<a href="https://colab.research.google.com/github/VasrshiniLekkala/FMML-LABS-/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?


Yes, **averaging the validation accuracy across multiple splits** does typically give **more consistent and reliable results**, especially when you're working with limited data or trying to get a better estimate of your model's true performance. This technique is commonly used in **cross-validation**, which helps reduce the impact of random variations or biases that might occur due to a single train-test split.

Here’s why **averaging** across multiple splits helps:

### 1. **Reduces Variance in Model Evaluation**
   - **Single Split**: If you only use a single train-test split, the performance metrics (e.g., validation accuracy) can be heavily influenced by the specific data points in your test set. If your test set happens to be easy or difficult (e.g., it contains mostly one class or very challenging examples), the reported accuracy might not reflect the model's true generalization ability.
   - **Multiple Splits**: When you use multiple splits (e.g., in **k-fold cross-validation**), you train and evaluate the model on different subsets of the data. Each split provides a different validation set, so the final reported validation accuracy is an average across all splits. This averaging helps smooth out the fluctuations caused by particular subsets, leading to a more robust estimate of model performance.

   - **Example**: In a 5-fold cross-validation, the data is divided into 5 parts. The model is trained and tested 5 times, each time using a different fold as the validation set, and the average accuracy from those 5 runs is computed. If one fold had particularly difficult examples, it wouldn’t disproportionately affect the final estimate.

### 2. **Minimizes the Risk of Overfitting**
   - If you train and test on the same single split, there is always the risk that your model could be overfitting to the specific data in the training set or the test set. Cross-validation helps ensure that the model is evaluated on different data points, and that performance is consistent across different data configurations.
   - The variability of the results across different splits can also highlight whether the model is truly generalizing or just memorizing the training data.

### 3. **Improves Robustness of Results**
   - **Consistency**: Averaging validation accuracy across multiple splits gives you a more consistent estimate of how well the model is expected to perform on unseen data.
   - **Better Generalization**: The more different subsets of the data the model is tested on, the better we can assess its ability to generalize. This reduces the chance that your result is a fluke caused by a particularly favorable or unfavorable test set.

### 4. **Helps with Model Selection**
   - When you are comparing different models, hyperparameters, or algorithms, using multiple splits and averaging the results allows you to get a better sense of which model is likely to perform best in a real-world scenario. It prevents you from overfitting to the performance on a single train-test split, which can lead to overly optimistic or pessimistic conclusions.

### 5. **Considerations for Averaging**
   - **Number of Splits (k in k-fold)**: Typically, **k=5 or k=10** is used for cross-validation. The number of splits can affect both the bias and variance of the estimated performance:
     - **Smaller k (e.g., k=5)**: More training data per split, but higher variance in the evaluation (due to fewer test samples).
     - **Larger k (e.g., k=10)**: More stable evaluation but fewer training samples per split, which can slightly reduce model performance due to less training data per fold.
   - **Stratified Cross-Validation**: For **classification** tasks, using **stratified k-fold cross-validation** ensures that the distribution of classes is similar in each fold, which can lead to more consistent results, especially when dealing with imbalanced datasets.
   - **Model Variability**: If your model’s performance fluctuates significantly across folds, it could indicate that the model is very sensitive to the particular data in each fold, which might suggest the need for better regularization or more data.

### 6. **Example: Cross-Validation vs. Single Split**
   - Let’s say you run an experiment where you evaluate your model on a single train-test split and get an accuracy of 85%. If you run 5-fold cross-validation and get the following accuracies:
     - Fold 1: 84%
     - Fold 2: 87%
     - Fold 3: 88%
     - Fold 4: 83%
     - Fold 5: 86%
   - The **average accuracy** across all folds is:
     \[
     \frac{84 + 87 + 88 + 83 + 86}{5} = 85.6\%
     \]
   - In this case, the cross-validation result (85.6%) gives a slightly more reliable estimate of your model's performance compared to the single split (85%), and the variability across folds (±2-3%) can help you assess how stable the model's performance is.

---

### Conclusion

Averaging validation accuracy across multiple splits **does provide more consistent results** because it reduces the impact of random fluctuations in the data (such as specific train-test splits), giving a more robust and reliable estimate of your model’s performance. This is especially important when:
- Your dataset is small.
- You need to avoid overfitting to a particular subset of data.
- You're comparing different models, hyperparameters, or features.

Therefore, using **cross-validation** (e.g., **k-fold cross-validation**) and averaging the validation accuracy (or other metrics) helps give a clearer, more stable understanding of how your model will perform on unseen data, leading to better generalization and more trustworthy results.



2. Does it give more accurate estimate of test accuracy?\

Averaging the validation accuracy across multiple splits (such as in **k-fold cross-validation**) generally **provides a better estimate of your model's expected test accuracy**, but it's important to clarify what this means and how it works.

### 1. **Validation Accuracy vs. Test Accuracy**
   - **Validation Accuracy**: This is the accuracy you get on the validation set during training, which is separate from the training data. It helps to evaluate how well the model generalizes to unseen data during the training process.
   - **Test Accuracy**: This is the accuracy of the model on a completely held-out test set, which the model has never seen during training. It's a more final and realistic measure of how well your model will perform on new, unseen data after training.

When you use **cross-validation**, you're not directly measuring the model's test accuracy, but you're using **validation sets** as proxies for how well the model might perform on unseen data. However, the key question is: **Does averaging the validation accuracy across multiple splits provide a better estimate of test accuracy?**

### 2. **How Cross-Validation Helps Estimate Test Accuracy**
   - **Better Representation of Data**: In k-fold cross-validation, the model is trained on different subsets of the data and validated on the remaining part. By doing this for multiple splits, the model is exposed to various parts of the data. This gives a better idea of how well the model can generalize across different samples of the data, which is closer to the real-world scenario where you have a test set it has never seen before.
   - **Averaging Reduces Variance**: If you evaluate a model on a single train-test split, the test accuracy might be subject to the specific characteristics of that split (e.g., an unusually easy or hard test set). Averaging across multiple validation sets reduces the variability and provides a more **robust estimate** of the model's performance.
   
   **Example**:
   - In a single test split, the model might perform very well or poorly depending on how the data is partitioned.
   - In k-fold cross-validation, the model is tested on various "folds," which makes the final averaged validation accuracy a more stable and consistent estimate.

### 3. **Bias and Variance in the Estimate**
   - **Bias**: Cross-validation can help reduce the **bias** in your estimate of model performance. If your single train-test split happens to have an unusually hard or easy test set, the reported performance will be biased (either overestimating or underestimating the model's true ability). Cross-validation smooths out such biases.
   
   - **Variance**: Cross-validation also reduces the **variance** in the estimate. A single split might result in a performance estimate that is too high or too low due to the specific data chosen for testing. Averaging over multiple folds gives you a **less variable** estimate, as each fold gives you a slightly different test set.

### 4. **Does Averaging Provide a Better Estimate of Test Accuracy?**
   - **Yes, to an extent.** Averaging over multiple validation sets (in cross-validation) gives you a better estimate of the model’s performance than relying on a single train-test split. However, it’s important to understand that **cross-validation gives you an estimate of how well the model would perform on unseen data** (which can be thought of as an approximation of test accuracy), but it's still not **the same as** evaluating the model on a completely separate test set.
   - If you use cross-validation on the training data and then evaluate the model on a completely separate **hold-out test set**, the final test accuracy might be a bit lower than the average cross-validation accuracy. This is because:
     - The model has been tuned on the training data (via the validation sets in cross-validation), and evaluating it on an unseen test set is the final **real-world estimate** of performance.

### 5. **Why Cross-Validation Might Overestimate Performance**
   Even though cross-validation gives a **better estimate** of how the model might perform on unseen data, it can still slightly **overestimate** test performance:
   - **Model Tuning**: If you're performing hyperparameter tuning based on cross-validation, you're essentially selecting the best hyperparameters from the validation splits. This means you're optimizing the model on data that the model has already seen (even though it's not directly trained on it), which could lead to some degree of **optimism** in the performance estimate.
   - **Final Test Set**: The **test set**, which is typically used after cross-validation to report the final performance, is usually untouched during the entire training process (including cross-validation). This is a more "real-world" scenario, and performance on this set is a more honest reflection of how the model will perform in production.

### 6. **Practical Advice**
   - **Use Cross-Validation for Model Selection**: Use k-fold cross-validation to estimate model performance and tune hyperparameters, but always keep a separate test set to evaluate the final model.
   - **Final Test Accuracy**: Once you’ve completed the cross-validation process and selected the final model (with tuned hyperparameters), evaluate it on the **test set** to get an unbiased estimate of its performance. This gives you a more accurate, final assessment of its real-world performance.

---

### Summary

- **Averaging the validation accuracy across multiple splits (cross-validation)** **reduces bias and variance**, giving a more **reliable estimate** of how well the model will perform on unseen data (which is closer to what you’d expect from the test set).
- It’s generally **better than relying on a single train-test split** because it accounts for different subsets of the data and avoids overfitting to a particular partition.
- However, **cross-validation still doesn't directly measure the test accuracy**, and it might slightly **overestimate** the model’s performance because the data used for validation is not entirely independent of the training data. The most accurate final test performance estimate comes from evaluating the model on a completely **separate test set** that was not involved in any training or validation.

Thus, while cross-validation gives a more consistent and robust estimate, the **final test accuracy** will still be the best true indicator of model performance, and this is ideally obtained from a **hold-out test set**.



3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

The number of **iterations** in a machine learning process (such as training or cross-validation) can certainly influence the **stability and accuracy** of the performance estimates. However, the effect depends on the **context** in which you're using the term "iterations." Generally, the two main ways "iterations" are used in machine learning are:

1. **Training Iterations (Epochs)** – The number of iterations during the training process, where each iteration corresponds to a step the model takes to improve its parameters.
2. **Cross-Validation Iterations** – The number of splits or folds used in k-fold cross-validation (e.g., using 5-fold or 10-fold cross-validation).

Let’s break down the effect of each in turn:

### 1. **Effect of More Training Iterations (Epochs)**
   The number of training iterations (or epochs) refers to how many times the model sees the entire dataset during training. More iterations allow the model to "learn" the data better, but there are diminishing returns after a certain point.

   **Key Effects of More Training Iterations:**

   - **Better Fit (Up to a Point)**:
     - In most machine learning algorithms (especially neural networks or gradient-based models), more training iterations can help the model learn more from the data, potentially improving performance. The model improves incrementally as it refines its parameters.
     - **Early Stopping**: Often, models will begin to "overfit" if trained for too many iterations, especially when the model is complex or the dataset is small. This is where techniques like **early stopping** come in to avoid overfitting by stopping training when the validation loss starts increasing.
   
   - **Diminishing Returns**:
     - At some point, continuing training for additional iterations leads to minimal improvements, and it can start to **overfit** to the training data, capturing noise rather than general patterns.
     - For simpler models, like linear regression, the model may reach an optimal solution quickly, and additional iterations won’t significantly improve performance.

   - **Effect on Performance Estimates**:
     - If you don’t train long enough (underfitting), your model might not capture the data's underlying patterns, resulting in **poor performance** and **biased estimates** of its ability.
     - Conversely, too many iterations without validation might lead to overfitting, which can **inflate the performance estimates** on training data but result in **poor generalization** on new, unseen data.

   - **Key Takeaway**:
     - More training iterations generally result in a **better fit** to the data, but only up to a point. After that, performance plateaus or deteriorates due to overfitting. The goal is to find the right balance where the model generalizes well without memorizing the data.

### 2. **Effect of More Cross-Validation Iterations (More Folds in k-Fold Cross-Validation)**
   The number of **cross-validation folds** (iterations in the context of validation) refers to how many subsets the data is divided into during **k-fold cross-validation**. For example, in 5-fold cross-validation, the dataset is split into 5 parts, and the model is trained and validated 5 times, each time using a different subset as the validation set.

   **Key Effects of More Cross-Validation Iterations:**

   - **More Consistent and Stable Estimates**:
     - **More folds** (e.g., using 10-fold instead of 5-fold) usually leads to a more **stable** and **reliable estimate** of the model’s performance, as it reduces the **variance** of the evaluation. This is particularly important when your dataset is small, as each fold will be trained on more data and tested on a slightly different subset.
     - With more folds, you are averaging over more validation sets, which can help smooth out any fluctuations in performance caused by random variability in the data.

   - **Smaller Variance**:
     - Using more folds can reduce the **variance** in performance estimates because the model gets to see different splits of the data. If one fold has particularly hard or easy data, the effects are averaged out across many folds. This makes your estimate less sensitive to peculiarities in any single train-test split.
     - For instance, **10-fold cross-validation** will give you a more stable estimate than **5-fold cross-validation**, because you are averaging over more splits.

   - **Computational Cost**:
     - The downside of more iterations (more folds) is the **increased computational cost**. More splits mean more models need to be trained, and thus it requires more time and resources.
     - For example, 10-fold cross-validation will involve training the model 10 times, while 5-fold would only require 5.

   - **Effect on Performance Estimates**:
     - **More folds** provide more consistent performance estimates by reducing the likelihood that your estimate is skewed due to the randomness in any single fold. However, the improvement is marginal after a certain number of folds. The difference between **5-fold** and **10-fold** cross-validation may not be substantial in most cases.

   - **Key Takeaway**:
     - Using more folds in cross-validation generally provides a **more reliable** estimate of model performance because it averages over more
FMML_M1L2.ipynb
FMML_M1L2.ipynb_
Cannot save changes
Machine Learning terms and metrics
FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.


[ ]
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

[ ]
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)
.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bureau publishes sample data (a block group typically has a population
of 600 to 3,000 people).

A household is a group of people residing within a home. Since the average
number of rooms and bedrooms in this dataset are provided per household, these
columns may take surprisingly large values for block groups with few households
and many empty houses, such as vacation resorts.

It can be downloaded/loaded using the
:func:`sklearn.datasets.fetch_california_housing` function.

.. topic:: References

    - Pace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,
      Statistics and Probability Letters, 33 (1997) 291-297

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.


[ ]
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)
Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)
The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.


[ ]
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel
We will also define a 'random classifier', which randomly allots labels to each sample


[ ]
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel
We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.


[ ]
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)
Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.


[ ]
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label
We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments


[ ]
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")
Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %
Experiments with splits
Let us reserve some of our train data as a validation set


[ ]
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
What is the accuracy of our classifiers on the train dataset?


[ ]
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")
Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %
For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.


[ ]
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")
Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %
Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.


[ ]
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")
Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %
You can run the above cell multiple times to try with different random splits. We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.


[ ]
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")
Test accuracy: 34.91795366795367 %
Try it out for yourself and answer:
How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?
Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using plt.plot. Check also for extreme values for splits, like 99.9% or 0.1%

Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

Multiple Splits
One way to get more accurate estimates for the test accuracy is by using cross-validation. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.


[ ]
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

[ ]
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")
Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %
So, **yes**, you typically get a **better estimate of test accuracy** with more **cross-validation folds** (iterations) because it averages over more splits, but for **training iterations**, the benefit is more nuanced, as the model might overfit if trained too long. The key is to balance these factors based on your dataset and model complexity.


4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check Cross-validatioin (Wikipedia)

Questions
Does averaging the validation accuracy across multiple splits give more consistent results?
Yes, averaging the validation accuracy across multiple splits does typically give more consistent and reliable results, especially when you're working with limited data or trying to get a better estimate of your model's true performance. This technique is commonly used in cross-validation, which helps reduce the impact of random variations or biases that might occur due to a single train-test split.

Here’s why averaging across multiple splits helps:

1. Reduces Variance in Model Evaluation
Single Split: If you only use a single train-test split, the performance metrics (e.g., validation accuracy) can be heavily influenced by the specific data points in your test set. If your test set happens to be easy or difficult (e.g., it contains mostly one class or very challenging examples), the reported accuracy might not reflect the model's true generalization ability.

Multiple Splits: When you use multiple splits (e.g., in k-fold cross-validation), you train and evaluate the model on different subsets of the data. Each split provides a different validation set, so the final reported validation accuracy is an average across all splits. This averaging helps smooth out the fluctuations caused by particular subsets, leading to a more robust estimate of model performance.

Example: In a 5-fold cross-validation, the data is divided into 5 parts. The model is trained and tested 5 times, each time using a different fold as the validation set, and the average accuracy from those 5 runs is computed. If one fold had particularly difficult examples, it wouldn’t disproportionately affect the final estimate.

2. Minimizes the Risk of Overfitting
If you train and test on the same single split, there is always the risk that your model could be overfitting to the specific data in the training set or the test set. Cross-validation helps ensure that the model is evaluated on different data points, and that performance is consistent across different data configurations.
The variability of the results across different splits can also highlight whether the model is truly generalizing or just memorizing the training data.
3. Improves Robustness of Results
Consistency: Averaging validation accuracy across multiple splits gives you a more consistent estimate of how well the model is expected to perform on unseen data.
Better Generalization: The more different subsets of the data the model is tested on, the better we can assess its ability to generalize. This reduces the chance that your result is a fluke caused by a particularly favorable or unfavorable test set.
4. Helps with Model Selection
When you are comparing different models, hyperparameters, or algorithms, using multiple splits and averaging the results allows you to get a better sense of which model is likely to perform best in a real-world scenario. It prevents you from overfitting to the performance on a single train-test split, which can lead to overly optimistic or pessimistic conclusions.
5. Considerations for Averaging
Number of Splits (k in k-fold): Typically, k=5 or k=10 is used for cross-validation. The number of splits can affect both the bias and variance of the estimated performance:
Smaller k (e.g., k=5): More training data per split, but higher variance in the evaluation (due to fewer test samples).
Larger k (e.g., k=10): More stable evaluation but fewer training samples per split, which can slightly reduce model performance due to less training data per fold.
Stratified Cross-Validation: For classification tasks, using stratified k-fold cross-validation ensures that the distribution of classes is similar in each fold, which can lead to more consistent results, especially when dealing with imbalanced datasets.
Model Variability: If your model’s performance fluctuates significantly across folds, it could indicate that the model is very sensitive to the particular data in each fold, which might suggest the need for better regularization or more data.
6. Example: Cross-Validation vs. Single Split
Let’s say you run an experiment where you evaluate your model on a single train-test split and get an accuracy of 85%. If you run 5-fold cross-validation and get the following accuracies:
Fold 1: 84%
Fold 2: 87%
Fold 3: 88%
Fold 4: 83%
Fold 5: 86%
The average accuracy across all folds is: [ \frac{84 + 87 + 88 + 83 + 86}{5} = 85.6% ]
In this case, the cross-validation result (85.6%) gives a slightly more reliable estimate of your model's performance compared to the single split (85%), and the variability across folds (±2-3%) can help you assess how stable the model's performance is.
Conclusion
Averaging validation accuracy across multiple splits does provide more consistent results because it reduces the impact of random fluctuations in the data (such as specific train-test splits), giving a more robust and reliable estimate of your model’s performance. This is especially important when:

Your dataset is small.
You need to avoid overfitting to a particular subset of data.
You're comparing different models, hyperparameters, or features.
Therefore, using cross-validation (e.g., k-fold cross-validation) and averaging the validation accuracy (or other metrics) helps give a clearer, more stable understanding of how your model will perform on unseen data, leading to better generalization and more trustworthy results.

Does it give more accurate estimate of test accuracy?\
Averaging the validation accuracy across multiple splits (such as in k-fold cross-validation) generally provides a better estimate of your model's expected test accuracy, but it's important to clarify what this means and how it works.

1. Validation Accuracy vs. Test Accuracy
Validation Accuracy: This is the accuracy you get on the validation set during training, which is separate from the training data. It helps to evaluate how well the model generalizes to unseen data during the training process.
Test Accuracy: This is the accuracy of the model on a completely held-out test set, which the model has never seen during training. It's a more final and realistic measure of how well your model will perform on new, unseen data after training.
When you use cross-validation, you're not directly measuring the model's test accuracy, but you're using validation sets as proxies for how well the model might perform on unseen data. However, the key question is: Does averaging the validation accuracy across multiple splits provide a better estimate of test accuracy?

2. How Cross-Validation Helps Estimate Test Accuracy
Better Representation of Data: In k-fold cross-validation, the model is trained on different subsets of the data and validated on the remaining part. By doing this for multiple splits, the model is exposed to various parts of the data. This gives a better idea of how well the model can generalize across different samples of the data, which is closer to the real-world scenario where you have a test set it has never seen before.

Averaging Reduces Variance: If you evaluate a model on a single train-test split, the test accuracy might be subject to the specific characteristics of that split (e.g., an unusually easy or hard test set). Averaging across multiple validation sets reduces the variability and provides a more robust estimate of the model's performance.

Example*:

In a single test split, the model might perform very well or poorly depending on how the data is partitioned.

In k-fold cross-validation, the model is tested on various "folds," which makes the final averaged validation accuracy a more stable and consistent estimate.

3. Bias and Variance in the Estimate
Bias: Cross-validation can help reduce the bias in your estimate of model performance. If your single train-test split happens to have an unusually hard or easy test set, the reported performance will be biased (either overestimating or underestimating the model's true ability). Cross-validation smooths out such biases.

Variance: Cross-validation also reduces the variance in the estimate. A single split might result in a performance estimate that is too high or too low due to the specific data chosen for testing. Averaging over multiple folds gives you a less variable estimate, as each fold gives you a slightly different test set.

4. Does Averaging Provide a Better Estimate of Test Accuracy?
Yes, to an extent. Averaging over multiple validation sets (in cross-validation) gives you a better estimate of the model’s performance than relying on a single train-test split. However, it’s important to understand that cross-validation gives you an estimate of how well the model would perform on unseen data (which can be thought of as an approximation of test accuracy), but it's still not the same as evaluating the model on a completely separate test set.
If you use cross-validation on the training data and then evaluate the model on a completely separate hold-out test set, the final test accuracy might be a bit lower than the average cross-validation accuracy. This is because:
The model has been tuned on the training data (via the validation sets in cross-validation), and evaluating it on an unseen test set is the final real-world estimate of performance.
5. Why Cross-Validation Might Overestimate Performance
Even though cross-validation gives a better estimate of how the model might perform on unseen data, it can still slightly overestimate test performance:

Model Tuning: If you're performing hyperparameter tuning based on cross-validation, you're essentially selecting the best hyperparameters from the validation splits. This means you're optimizing the model on data that the model has already seen (even though it's not directly trained on it), which could lead to some degree of optimism in the performance estimate.
Final Test Set: The test set, which is typically used after cross-validation to report the final performance, is usually untouched during the entire training process (including cross-validation). This is a more "real-world" scenario, and performance on this set is a more honest reflection of how the model will perform in production.
6. Practical Advice
Use Cross-Validation for Model Selection: Use k-fold cross-validation to estimate model performance and tune hyperparameters, but always keep a separate test set to evaluate the final model.
Final Test Accuracy: Once you’ve completed the cross-validation process and selected the final model (with tuned hyperparameters), evaluate it on the test set to get an unbiased estimate of its performance. This gives you a more accurate, final assessment of its real-world performance.
Summary
Averaging the validation accuracy across multiple splits (cross-validation) reduces bias and variance, giving a more reliable estimate of how well the model will perform on unseen data (which is closer to what you’d expect from the test set).
It’s generally better than relying on a single train-test split because it accounts for different subsets of the data and avoids overfitting to a particular partition.
However, cross-validation still doesn't directly measure the test accuracy, and it might slightly overestimate the model’s performance because the data used for validation is not entirely independent of the training data. The most accurate final test performance estimate comes from evaluating the model on a completely separate test set that was not involved in any training or validation.
Thus, while cross-validation gives a more consistent and robust estimate, the final test accuracy will still be the best true indicator of model performance, and this is ideally obtained from a hold-out test set.

What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
The number of iterations in a machine learning process (such as training or cross-validation) can certainly influence the stability and accuracy of the performance estimates. However, the effect depends on the context in which you're using the term "iterations." Generally, the two main ways "iterations" are used in machine learning are:

Training Iterations (Epochs) – The number of iterations during the training process, where each iteration corresponds to a step the model takes to improve its parameters.
Cross-Validation Iterations – The number of splits or folds used in k-fold cross-validation (e.g., using 5-fold or 10-fold cross-validation).
Let’s break down the effect of each in turn:

1. Effect of More Training Iterations (Epochs)
The number of training iterations (or epochs) refers to how many times the model sees the entire dataset during training. More iterations allow the model to "learn" the data better, but there are diminishing returns after a certain point.

Key Effects of More Training Iterations:

Better Fit (Up to a Point):

In most machine learning algorithms (especially neural networks or gradient-based models), more training iterations can help the model learn more from the data, potentially improving performance. The model improves incrementally as it refines its parameters.
Early Stopping: Often, models will begin to "overfit" if trained for too many iterations, especially when the model is complex or the dataset is small. This is where techniques like early stopping come in to avoid overfitting by stopping training when the validation losssplits, leading to a more stable result. However, the benefit of increasing folds diminishes after a certain point (often around 10 folds), and the **computational cost** increases.

### 3. **Does More Iterations Give a Better Estimate of Test Accuracy?**
   - **Training Iterations (Epochs)**: More iterations (epochs) allow the model to learn better from the data, but you risk overfitting after a certain number. The number of training iterations primarily affects the **quality** of the model itself, which indirectly impacts the quality of your performance estimates, but you should always check performance on a validation set to avoid overfitting.
   
   - **Cross-Validation Iterations (Folds)**: More folds generally give you **more consistent and stable estimates** of your model's expected test accuracy because you're averaging over more splits of the data. However, beyond a certain number of folds (typically 10), the improvement in accuracy stability becomes marginal, and the computational cost increases.

### 4. **Practical Tips**
   - For training iterations, you should monitor **validation performance** and use techniques like **early stopping** to avoid overfitting.
   - For cross-validation, **5-fold** or **10-fold** cross-validation is commonly used. **5-fold** is often sufficient for most datasets, but if you have a small dataset, you might consider **10-fold** for more stability. Too many folds can increase computational time without providing significant additional benefit.

---

### Summary

- **More Training Iterations (Epochs)**:
  - Generally improves model performance up to a point, but too many iterations can cause overfitting, leading to **inflated performance estimates** on training data and poor generalization to test data.
  - You need to balance **underfitting** (too few iterations) and **overfitting** (too many iterations).

- **More Cross-Validation Iterations (Folds)**:
  - Using more folds provides **more reliable and stable performance estimates** by reducing variance, especially for small datasets.
  - After about **10 folds**, the additional benefit of more folds becomes marginal, and the **computational cost** increases significantly.

So, **yes**, you typically get a **better estimate of test accuracy** with more **cross-validation folds** (iterations) because it averages over more splits, but for **training iterations**, the benefit is more nuanced, as the model might overfit if trained too long. The key is to balance these factors based on your dataset and model complexity.


4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


When dealing with a very small training or validation dataset, increasing the number of iterations (or epochs) may help the model to better fit the data, but it is not a guaranteed or always optimal solution. Here’s why:

### Training Dataset Size:
- **Overfitting Risk**: When the training dataset is small, the model might memorize the data during training, especially if you increase the number of iterations significantly. This can lead to overfitting, where the model performs well on the training data but poorly on unseen data (like the validation set).
- **Diminishing Returns**: With a very small dataset, the model may reach a point where additional iterations don’t improve its ability to generalize. Instead, it may just continue to memorize the limited examples, which reduces the ability to generalize to new, unseen data.

### Validation Dataset Size:
- **Validation as a Checkpoint**: A small validation set might not provide reliable feedback on the model’s performance, regardless of how many iterations you run. The validation performance might fluctuate wildly due to the small sample size, leading to misleading conclusions about model improvement or the need for more training.
  
### Alternative Approaches to Handle Small Datasets:
1. **Regularization**: Techniques like L2 regularization (weight decay), dropout, or data augmentation can help mitigate overfitting. These techniques force the model to generalize better, especially when data is scarce.
  
2. **Cross-validation**: If possible, use k-fold cross-validation on your small dataset to get a more reliable estimate of the model's performance. This helps in reducing the variability of results from a small validation set.
  
3. **Transfer Learning**: For very small datasets, using a pre-trained model and fine-tuning it on your data is often a more effective strategy than training from scratch. This leverages knowledge learned from large datasets and adapts it to your smaller dataset.
  
4. **Synthetic Data or Augmentation**: If you're working with image, text, or time-series data, data augmentation (e.g., rotations, translations for images or paraphrasing for text) can artificially expand the training set, which might allow you to run more iterations without overfitting.

5. **Early Stopping**: Rather than blindly increasing iterations, implement early stopping, which halts training when the validation performance starts to degrade. This helps prevent overfitting by ensuring that the model doesn't train for too many iterations on the small dataset.

### Summary:
- **Increasing iterations** can improve performance up to a point but might also lead to overfitting with a small dataset.
- **Regularization**, **cross-validation**, **transfer learning**, and **data augmentation** are often more effective strategies for small datasets.
- Focus on preventing overfitting, as a small dataset means the model will have fewer opportunities to learn generalizable patterns.

So, while increasing iterations might help in some cases, it’s not the best sole strategy for small datasets, and caution is needed to avoid overfitting.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.